# Python Libs

In [43]:
import gluoncv as gcv
import mxnet as mx
# mxnet NDarray 
from mxnet import nd

# MXNet Gluon Neural network layers.
from mxnet.gluon import nn

# optional for displaying the image
import matplotlib.pyplot as plt
import numpy as np
import os
import math

# Rotate Matrix

In [44]:
def build_rot_matrix(angle):
    #  a  b
    #  c  d
    
    # calculate terms
    a = math.cos(x)
    b = -math.sin(x)
    c = math.sin(x)
    d = math.cos(x)

    # print("a = ", a)
    # print("b = ", b)
    # print("c = ", c)
    # print("d = ", d)
    
    # build np array
    np_array = np.array([
        [a,b],
        [c,d]
    ])
    
    # build mx.nd array
    nd_array = mx.nd.array(np_array)
    
    return nd_array

### rotate 0 matrix

In [45]:
x=0

rotate_0 = build_rot_matrix(x)
print('rotate by 0 nd array is ', rotate_0)

rotate by 0 nd array is  
[[ 1. -0.]
 [ 0.  1.]]
<NDArray 2x2 @cpu(0)>


### rotate 90 matrix

In [46]:
x=0.5*math.pi

rotate_90 = build_rot_matrix(x)
print('rotate by 90 nd array is ', rotate_90)

rotate by 90 nd array is  
[[ 6.123234e-17 -1.000000e+00]
 [ 1.000000e+00  6.123234e-17]]
<NDArray 2x2 @cpu(0)>


### rotate 180/-180 matrix

In [47]:
x=math.pi

rotate_180 = build_rot_matrix(x)
print('rotate by 180 nd array is ', rotate_180)

rotate by 180 nd array is  
[[-1.0000000e+00 -1.2246469e-16]
 [ 1.2246469e-16 -1.0000000e+00]]
<NDArray 2x2 @cpu(0)>


### rotate 270 matrix

In [48]:
x=1.5*math.pi

rotate_270 = build_rot_matrix(x)
print('rotate by 270 nd array is ', rotate_270)

rotate by 270 nd array is  
[[-1.8369701e-16  1.0000000e+00]
 [-1.0000000e+00 -1.8369701e-16]]
<NDArray 2x2 @cpu(0)>


# Generic Rotate Routine

In [49]:
def my_print(verbose=False, *args):
    if (verbose):
        print(args)

In [62]:
def my_rotate(input_image_batch, rotate_matrix, verbose=False):
    # NDArrayIter(data, label=None, batch_size=1, shuffle=False, 
    #            last_batch_handle='pad', data_name='data', 
    #            label_name='softmax_label')
    #
    # Ignore the label parameter.
    dataiter = mx.io.NDArrayIter(input_image_batch, batch_size=1, shuffle=False, last_batch_handle='discard')
    for batch in dataiter:

        my_print(verbose, 'loop entry - a single batch - a single image in batch.data[0] from what is in in_data')
        
        
        # Does this copy or get an alias to the input image?
        a_img_batch = batch.data[0]
        my_print(verbose, 'a_img_batch = ', a_img_batch)
        my_print(verbose, 'a_img_batch.shape = ', a_img_batch.shape)

        #
        # Remember the dims so we can reshare the flattened array back to rectangular
        orig_dims = a_img_batch[0].shape
        
        
        a_img_indexes = mx.nd.contrib.index_array(a_img_batch, axes=(1, 2))
        my_print(verbose, 'a_img_indexes  ', a_img_indexes)
        my_print(verbose, 'a_img_indexes.shape ', a_img_indexes.shape)



        # Try to assign input data to output data based upon indicies
        #
        # Need to reshape so that rows=size of image sans batch
        # 2x2-> 4,2
        # 3x3-> 9,2
        #
        num_rows = a_img_batch[0].size
        my_print(True,'num_rows = ', num_rows)
        orig_indexes = mx.nd.reshape(a_img_indexes, shape=(num_rows,2))
        my_print(verbose, 'orig_indexes ', orig_indexes)
        orig_indexes = orig_indexes.astype("float32")

        # do the rotate
        new_indexes = nd.dot(orig_indexes, rotate_matrix) 
        my_print(verbose, 'new_indexes = ', new_indexes)
        new_indexes = new_indexes.astype('int64')
        my_print(verbose, 'new_indexes = ', new_indexes)
        #
        # shift to lower right quadrant. shift so that index axes is 0,0 in top left
        #
        # find the min row value
        min_row = new_indexes.min(axis=0)[0]
        my_print(verbose, 'min_row = ', min_row)
        adj_row = nd.abs(min_row).asscalar()
        my_print(verbose, 'adj_row = ', adj_row)
        
        # find the min col value
        min_col = new_indexes.min(axis=0)[1]
        my_print(verbose, 'min_col = ', min_col)
        adj_col = nd.abs(min_col).asscalar()
        my_print(verbose, 'adj_col = ', adj_col)
 
        # adjust based upon min row/col
        # with rotate 90 for 3x3 its [0,2]
        new_indexes = new_indexes + nd.array(nd.array([adj_row, adj_col])).astype('int64')
        my_print(verbose, 'new_indexes after shift to positive', new_indexes)

        #
        # The output image needs to be resized to match new dimensions.
        #
        
        #
        # Determine the max dim in array
        # 
        # find the min row value
        max_row = new_indexes.max(axis=0)[0].asscalar() + 1
        my_print(verbose, 'max_row = ', max_row)
        
        # find the min col value
        max_col = new_indexes.max(axis=0)[1].asscalar() + 1
        my_print(verbose, 'max_col = ', max_col)

        # This only works for square images rotated in 90 degree increments
        #output_image_batch = nd.zeros(input_image_batch.size).reshape(input_image_batch.shape)
        # Use new max col and max row
        output_image_batch = nd.zeros(input_image_batch.size).reshape([1, max_row, max_col])
        print('output_image_batch.shape = ', output_image_batch.shape)

        
        output_image = output_image_batch[0]
        
        my_print(verbose, 'output_image_batch ', output_image_batch)
        my_print(verbose, 'output_image_batch.shape ', output_image_batch.shape)
        my_print(verbose, 'output_image_batch[0] ', output_image_batch[0])
        print('works 0?')
        print('works 1?')
        print('a_img_batch[0].shape = ', a_img_batch[0].shape)
        print('new_indexes[:,0].max = ', new_indexes[:,0].max)
        print('new_indexes[:,1].max = ', new_indexes[:,1].max)
        my_print(verbose, 'a_img_batch[0][new_indexes[:,0],new_indexes[:,1]] ', a_img_batch[0][new_indexes[:,0],new_indexes[:,1]])
        #print(a_img_batch[0][new_indexes[:,0],0])
        #return 0
     
    
        output_image = a_img_batch[0][new_indexes[:,0],new_indexes[:,1]]
        my_print(verbose, 'output_image ', output_image)
        # output_image is flatened, need to resize to rectangular
        #
        #
        # FAILURE #1 
        # (new_dims, new_dims) = square_root(num_rows) only works when square
        # 
        # new_dims = float(num_rows)**0.5
        # new_dims = int(new_dims)
        # output_image = output_image.reshape(new_dims, new_dims)
        #
        # FAILURE #2
        # 
        # If the image is rotated so that H,W becomes W,H and W is greater than W, this will
        # fail since the indexes are out of range.
        #
        # output_image = output_image.reshape(orig_dims)
        
        
        # Hack
        #max_dim = max(max_col, max_row)
                
        output_image = output_image.reshape( max_row, max_col)    
        print('works 2?')
        
        my_print(verbose, 'output_image ', output_image)
        output_image_batch = nd.expand_dims(output_image, axis=0)
        my_print(verbose, 'output_image_batch ', output_image_batch)
        print('works 3?')
        
        return output_image_batch
        #return 0 # stub

# Test 90 

In [63]:
# Input Image 
in_img_batch = nd.arange(9).reshape((1,3,3))
print('in_img_batch is ', in_img_batch)
in_img = in_img_batch[0]
print('in_img ', in_img)

in_img_batch is  
[[[0. 1. 2.]
  [3. 4. 5.]
  [6. 7. 8.]]]
<NDArray 1x3x3 @cpu(0)>
in_img  
[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>


In [64]:
out_img_batch = my_rotate(in_img_batch, rotate_90)
out_img_batch

('num_rows = ', 9)


IndexError: Traceback (most recent call last):
  File "../src/operator/tensor/indexing_op.cc", line 454
IndexError: index 2 is out of bounds for axis 0 with size 2

# Test 0

In [65]:
out_img_batch = my_rotate(in_img_batch, rotate_0)
out_img_batch

('num_rows = ', 9)
output_image_batch.shape =  (1, 3, 3)
works 0?
works 1?
a_img_batch[0].shape =  (3, 3)
new_indexes[:,0].max =  <bound method NDArray.max of 
[0 0 0 1 1 1 2 2 2]
<NDArray 9 @cpu(0)>>
new_indexes[:,1].max =  <bound method NDArray.max of 
[0 1 2 0 1 2 0 1 2]
<NDArray 9 @cpu(0)>>
works 2?
works 3?



[[[0. 1. 2.]
  [3. 4. 5.]
  [6. 7. 8.]]]
<NDArray 1x3x3 @cpu(0)>

# Test 180

In [66]:
out_img_batch = my_rotate(in_img_batch, rotate_180)
out_img_batch

('num_rows = ', 9)
output_image_batch.shape =  (1, 3, 3)
works 0?
works 1?
a_img_batch[0].shape =  (3, 3)
new_indexes[:,0].max =  <bound method NDArray.max of 
[2 2 2 1 1 1 0 0 0]
<NDArray 9 @cpu(0)>>
new_indexes[:,1].max =  <bound method NDArray.max of 
[2 1 0 2 1 0 2 1 0]
<NDArray 9 @cpu(0)>>
works 2?
works 3?



[[[8. 7. 6.]
  [5. 4. 3.]
  [2. 1. 0.]]]
<NDArray 1x3x3 @cpu(0)>

# Test 270

In [67]:
# this fails.
#
# It should be
#
# 2 5 8
# 1 4 7
# 0 3 6
#
# I need to mirror in both axes
out_img_batch = my_rotate(in_img_batch, rotate_270, False)
out_img_batch

('num_rows = ', 9)
output_image_batch.shape =  (1, 3, 3)
works 0?
works 1?
a_img_batch[0].shape =  (3, 3)
new_indexes[:,0].max =  <bound method NDArray.max of 
[2 1 0 2 1 0 2 1 0]
<NDArray 9 @cpu(0)>>
new_indexes[:,1].max =  <bound method NDArray.max of 
[0 0 0 1 1 1 2 2 2]
<NDArray 9 @cpu(0)>>
works 2?
works 3?



[[[6. 3. 0.]
  [7. 4. 1.]
  [8. 5. 2.]]]
<NDArray 1x3x3 @cpu(0)>

# Test again for rotate 90 with a 2x2 

In [68]:
# Input Image 
in_img_batch = nd.arange(4).reshape((1,2,2))
print('in_img_batch is ', in_img_batch)
in_img = in_img_batch[0]
print('in_img ', in_img)

in_img_batch is  
[[[0. 1.]
  [2. 3.]]]
<NDArray 1x2x2 @cpu(0)>
in_img  
[[0. 1.]
 [2. 3.]]
<NDArray 2x2 @cpu(0)>


In [69]:
out_img_batch = my_rotate(in_img_batch, rotate_90)
out_img_batch

('num_rows = ', 4)
output_image_batch.shape =  (1, 2, 2)
works 0?
works 1?
a_img_batch[0].shape =  (2, 2)
new_indexes[:,0].max =  <bound method NDArray.max of 
[0 1 0 1]
<NDArray 4 @cpu(0)>>
new_indexes[:,1].max =  <bound method NDArray.max of 
[1 1 0 0]
<NDArray 4 @cpu(0)>>
works 2?
works 3?



[[[1. 3.]
  [0. 2.]]]
<NDArray 1x2x2 @cpu(0)>

# Test again for rotate 180 with a 2x2

In [70]:
out_img_batch = my_rotate(in_img_batch, rotate_180)
out_img_batch

('num_rows = ', 4)
output_image_batch.shape =  (1, 2, 2)
works 0?
works 1?
a_img_batch[0].shape =  (2, 2)
new_indexes[:,0].max =  <bound method NDArray.max of 
[1 1 0 0]
<NDArray 4 @cpu(0)>>
new_indexes[:,1].max =  <bound method NDArray.max of 
[1 0 1 0]
<NDArray 4 @cpu(0)>>
works 2?
works 3?



[[[3. 2.]
  [1. 0.]]]
<NDArray 1x2x2 @cpu(0)>

# Test with a 2x3 rotate_90

In [71]:
# Input Image 
in_img_batch = nd.arange(6).reshape((1,2,3))
print('in_img_batch is ', in_img_batch)
in_img = in_img_batch[0]
print('in_img ', in_img)

in_img_batch is  
[[[0. 1. 2.]
  [3. 4. 5.]]]
<NDArray 1x2x3 @cpu(0)>
in_img  
[[0. 1. 2.]
 [3. 4. 5.]]
<NDArray 2x3 @cpu(0)>


In [72]:
out_img_batch = my_rotate(in_img_batch, rotate_90)
out_img_batch

('num_rows = ', 6)
output_image_batch.shape =  (1, 3, 2)
works 0?
works 1?
a_img_batch[0].shape =  (2, 3)
new_indexes[:,0].max =  <bound method NDArray.max of 
[0 1 2 0 1 2]
<NDArray 6 @cpu(0)>>
new_indexes[:,1].max =  <bound method NDArray.max of 
[1 1 1 0 0 0]
<NDArray 6 @cpu(0)>>
works 2?
works 3?


IndexError: Traceback (most recent call last):
  File "../src/operator/tensor/indexing_op.cc", line 454
IndexError: index 2 is out of bounds for axis 0 with size 2